In [2]:
def kotak_bank(file_nme):
    import tabula
    from tabula import read_pdf
    import pandas as pd
    import numpy as np
    import re
    import operator


    def find_ifsc2(branch_details):
        ifsc_obt = ""
        t = branch_details.replace("-", ",").split(",")        # t[0]
        ifsc_found = dt.loc[operator.and_(operator.and_(dt["Name of the bank"] == "Kotak Mahindra Bank" , dt["Branch Name"].str.contains("{}".format(t[-1].strip(" ")))),dt["Address"].str.contains("{}".format(t[0].strip(" "))))]
        if "KKBK" in str(ifsc_found["IFSC Code"]):
            index = str(ifsc_found).find("KKBK")
            ifsc_obt = str(ifsc_found)[index: index+11]
        elif "Empty" in str(ifsc_found):
            print("Branch not found in the dataset!!")

        return ifsc_obt
    
    
    dt = pd.read_csv('IFSC codes1.csv', header=0)
    try:
        tables=read_pdf(file_nme, area=(15, 15, 260, 800), pages = 1)
        df2 = pd.DataFrame()
        df2 = pd.concat([c for c in tables]).drop_duplicates()
        df2.reset_index(drop=True, inplace=True)
        df2.dropna(axis=1, how="all", thresh=None, subset=None, inplace=True)
        df2 = pd.DataFrame(np.row_stack([df2.columns, df2.values]),columns=['1', '2', '3'])

        acc_n2 = ""; acc_no2 = ""; addr2 = ""; br_details2 = "";
        cifn2 = ""; ifsc2 = ""; micrn2 = "";

        cs_typ1 = False

        siz = len(df2.columns)
        for j in range(1, siz):
            for count,i in enumerate(df2[df2.columns[j]]):
                try:
                    if "account no" in i.lower():
                        cs_typ1 = True
                        if (i.lower().strip("account no").strip(" ").strip(".").strip(":").strip(" ")) != "":
                            acc_no2 = (i.lower().strip("account no").strip(" ").strip(".").strip(":").strip(" "))
                        else:
                            acc_no2 = (df2[df2.columns[j+1]][count])
                    if ("branch" in i.lower()) and cs_typ1:
                        if (i.lower().lstrip("branch").strip(" ").strip(".").strip(":").strip(" ")) != "":
                            br_details2 = (i.lower().lstrip("branch").strip(" ").strip(".").strip(":").strip(" "))
                        else:
                            br_details2 = (df2[df2.columns[j+1]][count]).strip(" ").strip(".").strip(":").strip(" ")   
                except AttributeError:
                    pass

        line = 0
        for count,i in enumerate(df2[df2.columns[0]]):
            try:
                if i.isupper():
                    line += 1
                    if line == 1:
                        acc_n2 = i
                    elif line > 1:
                        addr2 += i + " "
            except AttributeError:
                pass

        dfr2 = pd.DataFrame([["Account Name", acc_n2.upper()]], columns = ["Customer Information", "Data_Extracted"])
        dfr2.loc[len(dfr2)] = ["Account Number", acc_no2]
        if ifsc2 != "":
            dfr2.loc[len(dfr2)] = ["IFS Code", ifsc2]
        else:
            dfr2.loc[len(dfr2)] = ["IFS Code", find_ifsc2(br_details2)]
        dfr2.loc[len(dfr2)] = ["Branch Name", br_details2.upper()]
        dfr2.loc[len(dfr2)] = ["Address", addr2.upper()]
        dfr2.loc[len(dfr2)] = ["MICR Code", micrn2]
        dfr2.loc[len(dfr2)] = ["CIF Number", cifn2]

    except AssertionError:
        tables=read_pdf(file_nme, area=(15, 15, 160, 800), pages = 2)
        df2 = pd.DataFrame()
        df2 = pd.concat([c for c in tables]).drop_duplicates()
        df2.reset_index(drop=True, inplace=True)
        df2.dropna(axis=1, how="all", thresh=None, subset=None, inplace=True)
        
        acc_n2 = ""; acc_no2 = ""; addr2 = ""; br_details2 = "";
        cifn2 = ""; ifsc2 = ""; micrn2 = "";
        line = 0
        siz = len(df2.columns)

        for count,i in enumerate(df2[df2.columns[0]]):
            try:
                if i.isupper():
                    line += 1
                    if line == 1:
                        acc_n2 = i
                if "account no" in i.lower():
                    if (i.lower().strip("account no").strip(" ").strip(".").strip(":").strip(" ")) != "":
                        acc_no2 = (i.lower().strip("account no").strip(" ").strip(".").strip(":").strip(" "))
                    else:
                        acc_no2 = (df1[df1.columns[1]][count]).strip(" ").strip(".").strip(":").strip(" ")

            except AttributeError:
                pass


        for j in range(1, siz):
            for count,i in enumerate(df2[df2.columns[j]]):
                try:
                    if "home branch" in i.lower():
                        if (i.lower().strip("home branch").strip(" ").strip(".").strip(":").strip(" ")) != "":
                            br_details2 = (i.lower().strip("home branch").strip(" ").strip(".").strip(":").strip(" "))
                        else:
                            br_details2 = (df2[df2.columns[j+1]][count]).strip(" ").strip(":").strip(" ")
                    if "micr code" in i.lower():
                        if (i.lower().strip("micr code").strip(" ").strip(".").strip(":").strip(" ")) != "":
                            micrn2 = (i.lower().strip("micr code").strip(" ").strip(".").strip(":").strip(" "))
                        else:
                            micrn2 = (df2[df2.columns[j+1]][count]).strip(" ").strip(":").strip(" ")
                    if "ifsc code" in i.lower():
                        if (i.lower().strip("ifsc code").strip(" ").strip(".").strip(":").strip(" ")) != "":
                            ifsc2 = (i.lower().strip("ifsc code").strip(" ").strip(".").strip(":").strip(" "))
                        else:
                            ifsc2 = (df2[df2.columns[j+1]][count]).strip(" ").strip(":").strip(" ")
                except AttributeError:
                    pass
                

        dfr2 = pd.DataFrame([["Account Name", acc_n2]], columns = ["Customer Information", "Data_Extracted"])
        dfr2.loc[len(dfr2)] = ["Account Number", acc_no2]
        if ifsc2 != "":
            dfr2.loc[len(dfr2)] = ["IFS Code", ifsc2]
        else:
            dfr2.loc[len(dfr2)] = ["IFS Code", find_ifsc2(br_details2)]
        dfr2.loc[len(dfr2)] = ["Branch Name", br_details2]
        dfr2.loc[len(dfr2)] = ["Address", addr2]
        dfr2.loc[len(dfr2)] = ["MICR Code", micrn2]
        dfr2.loc[len(dfr2)] = ["CIF Number", cifn2]

    return dfr2

In [3]:
kotak_bank("kotak01.pdf")

/home/divyanshu/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3242: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
Got stderr: Apr 03, 2020 3:13:54 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Customer Information,Data_Extracted
0,Account Name,ADITI GHEI
1,Account Number,6812074192
2,IFS Code,KKBK0004611
3,Branch Name,"PANCHSHEEL PARK, NEW DELHI"
4,Address,S-304 PANCHSHEEL PARK NEW DELHI MALVIYA NAGAR ...
5,MICR Code,
6,CIF Number,
